In [1]:
import requests, lxml
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict

In [50]:
def get_article_text(_url):
    _response = requests.get(_url)
    _response.raise_for_status()
    _soup = BeautifulSoup(_response.content,'lxml')
    # use article for washingtonpost, and p for nytimes
    _text = ' '.join(map(lambda a: a.text, _soup.find_all('article')))\
                .encode('ascii', errors='replace') \
                .decode().replace("?"," ")
    return _text

In [51]:
def summarize(_text, _summary_lines):
    _sentences = sent_tokenize(_text)    
    assert _summary_lines <= len(_sentences)
    
    _stopwords = set(stopwords.words('english') + list(punctuation))

    _words = word_tokenize(_text.lower())    
    _words = [_word for _word in _words if _word not in _stopwords]
    _words_freq_dist = FreqDist(_words)
       
    _sentence_rankings = defaultdict(int)
    for _idx, _sentence in enumerate(_sentences):
        for _word in word_tokenize(_sentence.lower()):
            if _word in _words_freq_dist:
                _sentence_rankings[_idx] += _words_freq_dist[_word]            
        
    _summary = nlargest(_summary_lines, _sentence_rankings, key=_sentence_rankings.get)
    return [_sentences[_idx] for _idx in sorted(_summary)]

In [53]:
'''
_article_url = "https://www.washingtonpost.com/news/the-switch/wp/2016/10/18/the-pentagons-massive-new-telescope-is-designed-to-track-space-junk-and-watch-out-for-killer-asteroids/"
_article_url = 'https://www.washingtonpost.com/world/national-security/trump-weighing-military-options-following-chemical-weapons-attack-in-syria/2017/04/06/0c59603a-1ae8-11e7-9887-1a5314b56a08_story.html?hpid=hp_hp-banner-main_syria-315pm-banner%3Ahomepage%2Fstory&utm_term=.c5c336272fcc'
_article_url = "https://www.washingtonpost.com/news/energy-environment/wp/2017/04/05/carbon-dioxide-levels-could-reach-their-highest-point-in-50-million-years-by-the-end-of-the-century/?tid=hybrid_collaborative_1_na&utm_term=.c0f2be7313ef"
_article_url = 'https://www.nytimes.com/2017/04/07/world/middleeast/missile-strike-syria-russia.html?hp&action=click&pgtype=Homepage&clickSource=story-heading&module=span-ab-top-region&region=top-news&WT.nav=top-news&_r=0'
'''

_article_url = "https://www.washingtonpost.com/news/energy-environment/wp/2017/04/05/carbon-dioxide-levels-could-reach-their-highest-point-in-50-million-years-by-the-end-of-the-century/?tid=hybrid_collaborative_1_na&utm_term=.c0f2be7313ef"
_article_text = get_article_text(_article_url)
_summary = summarize(_article_text, 3)

for _idx, _sentence in enumerate(_summary):
    print('{}: {}\n'.format(_idx,_sentence))
    

0: (Kevin Frayer/Getty Images)  Continuing to burn fossil fuels at the current rate could bring atmospheric carbon dioxide to its highest concentration in 50 million years, jumping from about 400 parts per million now to more than 900 parts per million by the end of this century, a new study warns.

1: A business-as-usual trajectory suggests that carbon dioxide levels could exceed 2,000 parts per million by the year 2250, concentrations that were last seen about 200 million years ago.

2: Additionally, at least one study has suggested that concentrations could be as high as 5,000 parts per million by the year 2400 if humans were to burn through all the fossil fuels on Earth, and that would result in both the highest carbon dioxide levels and the highest temperatures seen throughout the study period.

